In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

In [6]:
bucket_name = 'bankdetail971267' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

ap-southeast-1


In [13]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'ap-southeast-1':
        s3.create_bucket(Bucket=bucket_name,
                                  CreateBucketConfiguration={
                                      'LocationConstraint': 'ap-southeast-1'}
                                  )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)


S3 bucket created successfully


In [14]:
# set an output path where the trained model will be saved
prefix = 'xgboost-built-in'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankdetail971267/xgboost-built-in/output


In [15]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve (
        "https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv"
                                , "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [16]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [17]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [18]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)

In [19]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
display(s3_input_train)

In [20]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

## Building Models Xgboot- Inbuilt Algorithm

In [21]:
container = get_image_uri(
    boto3.Session().region_name
    ,'xgboost'
    , repo_version='1.0-1'
)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [22]:
container = sagemaker.image_uris.retrieve(    
    'xgboost'
    , boto3.Session().region_name
    , version='1.0-1'
)

In [23]:
container

'121021644041.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3'

In [24]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [25]:
sagemaker.get_execution_role()

'arn:aws:iam::494026973566:role/service-role/AmazonSageMaker-ExecutionRole-20220816T134046'

In [26]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1,                                           
                                          instance_type='ml.m5.large', # 'ml.m5.2xlarge', t3.micro
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [27]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2022-08-16 08:27:43 Starting - Starting the training job...
2022-08-16 08:28:00 Starting - Preparing the instances for trainingProfilerReport-1660638463: InProgress
.........
2022-08-16 08:29:38 Downloading - Downloading input data...
2022-08-16 08:30:06 Training - Downloading the training image.....INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[08:30:56] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Determined delimiter of CSV input is ','
[08:30

In [28]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m5.large')

-----!

In [29]:
from sagemaker.serializers import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer(content_type='text/csv') # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [30]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [31]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [32]:
xgb_predictor.endpoint_name

'sagemaker-xgboost-2022-08-16-08-32-46-916'

In [33]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint_name)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'ZWZSS0N14VMBNNXB',
   'HostId': 'S4BYSc6vIyqFRPtEwx+H1HiIAgegqzNWkpT9TX3deojQsuGkyRvCYnJHiex9YaOh+Hh1YluGP0o=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'S4BYSc6vIyqFRPtEwx+H1HiIAgegqzNWkpT9TX3deojQsuGkyRvCYnJHiex9YaOh+Hh1YluGP0o=',
    'x-amz-request-id': 'ZWZSS0N14VMBNNXB',
    'date': 'Tue, 16 Aug 2022 08:36:15 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-built-in/output/sagemaker-xgboost-2022-08-16-08-27-43-793/profiler-output/system/incremental/2022081608/1660638540.algo-1.json'},
   {'Key': 'xgboost-built-in/output/sagemaker-xgboost-2022-08-16-08-27-43-793/profiler-output/system/training_job_end.ts'},
   {'Key': 'xgboost-built-in/output/sagemaker-xgboost-2022-08-16-08-27-43-793/rule-output/ProfilerReport-1660638463/profiler-output/profiler-reports/LoadBalancing.json'},
   {'Key': 'xg